<a href="https://colab.research.google.com/github/mangalm96/explainablenlp-info259/blob/main/process_tWTWT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install ekphrasis

     |████████████████████████████████| 80 kB 4.2 MB/s 
     |████████████████████████████████| 45 kB 2.6 MB/s 
     |████████████████████████████████| 53 kB 1.5 MB/s 
  Created wheel for ekphrasis: filename=ekphrasis-0.5.1-py3-none-any.whl size=82842 sha256=3792cc838131e19d047112a5daa8e8186682f019aa9f3a1b576ef93eb513b31b
  Stored in directory: /root/.cache/pip/wheels/f7/ec/0d/12659e32faf780546945d0120f2c8410eb3efb7426731da88f
Successfully built ekphrasis


In [3]:
import json
import os
import sys
import glob

import re
from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons

In [4]:
EMOJI_PATTERN = re.compile("["
         u"\U0001F600-\U0001F64F"  # emoticons
         u"\U0001F300-\U0001F5FF"  # symbols & pictographs
         u"\U0001F680-\U0001F6FF"  # transport & map symbols
         u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
         u"\U00002702-\U000027B0"
         u"\U000024C2-\U0001F251"
         "]+", flags=re.UNICODE)

In [5]:
text_processor = TextPreProcessor(
    normalize=['url', 'email', 'percent', 'money', 'phone',
        'time', 'url', 'date', 'number'],
    annotate={"hashtag", "allcaps", "elongated", "repeated",
        'emphasis', 'censored'},

    fix_html=True,
    segmenter="twitter",
    corrector="twitter",

    unpack_hashtags=True,
    unpack_contractions=True,
    spell_correct_elong=True,

    tokenizer=SocialTokenizer(lowercase=False).tokenize,
    dicts=[emoticons]
)


/usr/local/lib/python3.7/dist-packages/ekphrasis/classes/tokenizer.py:225: FutureWarning: Possible nested set at position 2190
  self.tok = re.compile(r"({})".format("|".join(pipeline)))


Word statistics files not found!
Downloading... done!
Unpacking... done!
Reading twitter - 1grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/twitter/counts_1grams.txt
Reading twitter - 2grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/twitter/counts_2grams.txt
Reading twitter - 1grams ...


/usr/local/lib/python3.7/dist-packages/ekphrasis/classes/exmanager.py:14: FutureWarning: Possible nested set at position 42
  regexes = {k.lower(): re.compile(self.expressions[k]) for k, v in


In [6]:
REMOVE_TAGS = [
    "<emphasis>", "<kiss>", "<repeated>", "<laugh>", "<allcaps>",
    "</allcaps>", "<angel>", "<elongated>", "<tong>", "<annoyed>",
    "<censored>", "<happy>", "<percent>", "<wink>",
    "<headdesk>", "<surprise>", "<date>", "<time>", "<url>",
    "<sad>", "<email>", "<phone>", "<hashtag>", "</hashtag>"
    ]

ADD_TO_GLOVE = ["<number>", "<money>"]

In [7]:
REMOVE_PUNCTUATIONS = ["!", "@", "#", "$", "%", "^", "&", "*", "(", ")", "-", "+", "_", "=", "{", "}", "[", "]", ":", ";", "'", '"', ",", ".", "?", "/"]

In [8]:
# Try removing punctuations as well

def pre_process_single(tweet, t_id):
    tweet_toked_text = []
    de_emojified_text = tweet.encode('ascii', 'ignore').decode('ascii')
    de_emojified_text = re.sub(r"\$", " $ ", de_emojified_text)
    company_normalize_text = EMOJI_PATTERN.sub(r' ', de_emojified_text)

    tokens = text_processor.pre_process_doc(company_normalize_text)
    for token in tokens:
        if token in REMOVE_TAGS:
            continue
        if token in REMOVE_PUNCTUATIONS: #remove punctuations
            continue
        tweet_toked_text.append(str(token))

    return tweet_toked_text

In [9]:
id2text = {}
tweet_acquired = 0
tweet_valid = 0
tweet_invalid = 0

In [13]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [14]:
from zipfile import ZipFile
with ZipFile("/content/gdrive/MyDrive/NLP Stance Detection/Code_Kaushal/data/wtwt/scrapped_full.zip", 'r') as zip:
    # printing all the contents of the zip file
    zip.printdir()
  
    # extracting all the files
    print('Extracting all the files now...')
    zip.extractall()
    print('Done!')

Streaming output truncated to the last 5000 lines.
scrapped_full/753825939184680960.json          2022-03-31 19:20:30          183
scrapped_full/939670654521085953.json          2022-04-01 04:28:14          287
scrapped_full/758726319731507200.json          2022-03-31 13:57:26          201
scrapped_full/928112839729938432.json          2022-04-01 12:39:08          191
scrapped_full/734822135479324672.json          2022-03-31 22:14:50          238
scrapped_full/807317714897399809.json          2022-03-31 10:51:40          239
scrapped_full/1042883538490732544.json         2022-03-31 05:10:56          385
scrapped_full/884266517029949444.json          2022-04-01 19:09:28          221
scrapped_full/935255760249982977.json          2022-03-30 16:57:08          287
__MACOSX/scrapped_full/._935255760249982977.json 2022-03-30 16:57:08          220
scrapped_full/509618809351573504.json          2022-03-31 09:44:20          231
scrapped_full/651116178618748928.json          2022-04-01 00:49:26 

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Streaming output truncated to the last 5000 lines.
scrapped_full/841669253958914050.json          2022-03-31 18:26:40          217
scrapped_full/937837942571241477.json          2022-03-31 02:18:56          239
__MACOSX/scrapped_full/._937837942571241477.json 2022-03-31 02:18:56          220
scrapped_full/761621288314023936.json          2022-03-31 14:14:04          228
scrapped_full/972283851035754497.json          2022-04-01 02:30:02          345
scrapped_full/1047589834766934016.json         2022-03-31 08:36:30          262
scrapped_full/481152243824357376.json          2022-03-31 16:45:54          191
scrapped_full/941770670064599040.json          2022-03-30 23:05:56          287
scrapped_full/949248288498307073.json          2022-04-01 14:03:38          152
scrapped_full/927662940060139521.json          2022-04-01 19:10:18          228
scrapped_full/616647095315906560.json          2022-03-31 18:10:18          228
scrapped_full/827231733535830017.json          2022-03-31 20:31:44 

In [ ]:
path = 'scrapped_full/'
files = os.listdir(path)
files # all files are here

In [23]:
files[0]

'731521826308947968.json'

In [29]:
for fil in files:
    file_name = path + fil
    # print(file_name)
    if fil!=".DS_Store":
      fo = open(file_name, "r")
      # print(fo)
      full_tweet = json.load(fo)
      fo.close()
      tweet_acquired = tweet_acquired + 1
      if "data" not in full_tweet:
          tweet_invalid = tweet_invalid + 1
          continue
      tweet_valid = tweet_valid + 1
      tweet_id = full_tweet["data"][0]["id"]
      txt = pre_process_single(full_tweet["data"][0]["text"], tweet_id)
      if len(txt) > 0 and txt != ["<user>"]:
          id2text[tweet_id] = txt
      else: 
          pass

In [30]:
print(f'the number of tweet acquired is:',{tweet_acquired})
print(f'the number of invalid tweet is:',{tweet_invalid})
print(f'the number of valid tweet is:',{tweet_valid})

the number of tweet acquired is: {225611}
the number of invalid tweet is: {32248}
the number of valid tweet is: {193363}


In [35]:
all_keys = id2text.keys()
len(all_keys)

43010

In [32]:
fo = open("/content/gdrive/MyDrive/NLP Stance Detection/Code_Kaushal/data/twt_wt/wtwt_new.json", "r")
wtwt = json.load(fo)
fo.close()

In [36]:
wtwt_obtained = []

In [37]:
merger2target = json.load(open('/content/gdrive/MyDrive/NLP Stance Detection/tWT_WT/dataset/merger2target.json'))

In [38]:
for data in wtwt:
    if data["tweet_id"] in all_keys:
        d = data.copy()
        d["target"] = merger2target[d["merger"]].split()
        d["text"] = id2text[data["tweet_id"]]
        d["merger"] = d["merger"][4:] if "NEG_" in d["merger"] else d["merger"]
        assert d["merger"] in ['CVS_AET', 'CI_ESRX', 'AET_HUM', 'ANTM_CI', 'DIS_FOX']
        wtwt_obtained.append(d)

In [39]:
fo = open("/content/gdrive/MyDrive/NLP Stance Detection/tWT_WT/dataset/data_new.json", "w+")
json.dump(wtwt_obtained, fo, indent=2)
fo.close()